In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
Image.LOAD_TRUNCATED_IMAGES = True
import multiprocessing
import shutil
from glob import glob

import numpy as np
from keras import backend
from keras.callbacks import ReduceLROnPlateau,LearningRateScheduler
from keras.callbacks import TensorBoard, Callback, ModelCheckpoint
from keras.layers import Dense, GlobalAveragePooling2D,Dropout
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras_radam import RAdam
from lookahead import Lookahead
import keras.backend as K

Using TensorFlow backend.


In [2]:
from classification_models.keras import Classifiers
xception, preprocess_input = Classifiers.get('xception')
# import efficientnet.keras as efn
# preprocess_input = efn.preprocess_input
# from keras_applications.efficientnet import EfficientNetB4,preprocess_input

In [3]:
from data_gen_label_cut  import data_flow
batch_size = 32
num_classes = 9
input_size = 380
train_sequence, validation_sequence = data_flow(batch_size,num_classes,input_size,preprocess_input)

total label: 10651 
total img: 10651 
total samples: 10651, training samples: 9585, validation samples: 1066


In [ ]:
from keras.utils import multi_gpu_model
def model_fn(objective, optimizer, metrics):
#     base_model = efn.EfficientNetB4(include_top=False,
    base_model = xception(include_top=False,
                           input_shape=(input_size, input_size, 3),
                           classes=num_classes,
                           weights='imagenet',)
    x = base_model.output
    x = GlobalAveragePooling2D( )(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model1 = Model(inputs=base_model.input, outputs=predictions)
    model2 = multi_gpu_model(model1, gpus=3)
    model2.compile(loss=objective, optimizer=optimizer, metrics=metrics)
    lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
    lookahead.inject(model2) # add into model
    model2.summary( )
   #     model.load_weights('./model_snapshots/G0003/weights_036_0.8510.h5')
  #     print("load ok!!!!!")
    return model1,model2
optimizer = RAdam(learning_rate=1e-3)
objective = 'categorical_crossentropy'
metrics = ['accuracy']
model1,model2 = model_fn(objective, optimizer, metrics)
print("model ok!")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 380, 380, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 380, 380, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 380, 380, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 380, 380, 3)  0           input_1[0][0]          

In [ ]:
class MyCbk(Callback):
    def __init__(self, model):
         self.model_to_save = model
    def on_epoch_end(self, epoch, logs=None):
        self.model_to_save.save('./model_snapshots/X0003/model_at_epoch_%d.h5' % epoch)

cbk1 = MyCbk(model1)

In [ ]:
log_local = './logs/'+'X0003-xception(380)-cutmix&mixup(1.0)-wp'
tensorBoard = TensorBoard(log_dir=log_local)

In [ ]:
from warmup_cosine_decay_scheduler import WarmUpCosineDecayScheduler
learning_rate = 1e-3
sample_count = len(train_sequence) * batch_size
epochs = 50
warmup_epoch = 8
batch_size = batch_size
learning_rate_base = learning_rate
total_steps = int(epochs * sample_count / batch_size)
warmup_steps = int(warmup_epoch * sample_count / batch_size)

warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=learning_rate_base,
                                        total_steps=total_steps,
                                        warmup_learning_rate=0,
                                        warmup_steps=warmup_steps,
                                        hold_base_rate_steps=0,
                                        )

In [ ]:
# def scheduler(epoch):
#     # 每隔100个epoch，学习率减小为原来的1/10
#     if epoch % 4 == 0 and epoch != 0:
#         lr = K.get_value(model2.optimizer.lr)
#         K.set_value(model2.optimizer.lr, lr * 0.8)
#         print("lr changed to {}".format(lr * 0.8))
#     return K.get_value(model2.optimizer.lr)
 
# reduce_lr = LearningRateScheduler(scheduler)

In [ ]:
model2.fit_generator(
    train_sequence,
    steps_per_epoch=len(train_sequence),
    epochs=150,
    verbose=1,
    callbacks=[cbk1, tensorBoard, warm_up_lr],
    validation_data=validation_sequence,
    max_queue_size=10,
    workers=int(multiprocessing.cpu_count( ) * 0.9),
    use_multiprocessing=True,
    shuffle=True
)

Epoch 1/150
300/300 [==============================] - 233s 777ms/step - loss: 2.0331 - acc: 0.3132 - val_loss: 1.3807 - val_acc: 0.6426
Epoch 2/150
300/300 [==============================] - 204s 679ms/step - loss: 1.6366 - acc: 0.5843 - val_loss: 0.9877 - val_acc: 0.8133
Epoch 3/150
300/300 [==============================] - 208s 693ms/step - loss: 1.5105 - acc: 0.6038 - val_loss: 0.8724 - val_acc: 0.8518
Epoch 4/150
300/300 [==============================] - 210s 699ms/step - loss: 1.4196 - acc: 0.6746 - val_loss: 0.9198 - val_acc: 0.8255
Epoch 5/150
300/300 [==============================] - 211s 704ms/step - loss: 1.3771 - acc: 0.6793 - val_loss: 0.9563 - val_acc: 0.8096
Epoch 6/150
300/300 [==============================] - 207s 691ms/step - loss: 1.3645 - acc: 0.7241 - val_loss: 0.8685 - val_acc: 0.8368
Epoch 7/150
300/300 [==============================] - 206s 686ms/step - loss: 1.3059 - acc: 0.7502 - val_loss: 0.8863 - val_acc: 0.8433
Epoch 8/150
300/300 [====================

300/300 [==============================] - 204s 680ms/step - loss: 1.0307 - acc: 0.7846 - val_loss: 0.8380 - val_acc: 0.8790
Epoch 61/150
300/300 [==============================] - 209s 696ms/step - loss: 1.0264 - acc: 0.7757 - val_loss: 0.8261 - val_acc: 0.8790
Epoch 62/150
300/300 [==============================] - 209s 696ms/step - loss: 0.9889 - acc: 0.8360 - val_loss: 0.8177 - val_acc: 0.8734
Epoch 63/150
300/300 [==============================] - 211s 703ms/step - loss: 1.0211 - acc: 0.8804 - val_loss: 0.8256 - val_acc: 0.8705
Epoch 64/150
300/300 [==============================] - 206s 686ms/step - loss: 0.9711 - acc: 0.8306 - val_loss: 0.7929 - val_acc: 0.8818
Epoch 65/150
300/300 [==============================] - 206s 687ms/step - loss: 0.9779 - acc: 0.8832 - val_loss: 0.8000 - val_acc: 0.8846
Epoch 66/150
300/300 [==============================] - 202s 673ms/step - loss: 1.0143 - acc: 0.8322 - val_loss: 0.8548 - val_acc: 0.8790
Epoch 67/150
300/300 [=========================